In [11]:
# # see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
# %load_ext autoreload
# %autoreload 2
# %matplotlib inline
# import pdb

In [12]:
import os
import time
import os.path as op
import argparse
from data.data_pipe import get_val_pair
from torchvision import transforms as trans
from tqdm import tqdm_notebook as tqdm
import torch
import numpy as np
from IPython.display import display

from Learner import face_learner
from config import get_config
from utils import hflip_batch, cosineDim1
from data.datasets import (IJBCroppedFacesDataset, IJBCVerificationPathDataset,
                           ARVerificationAllPathDataset, IJBAVerificationDataset)

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [13]:
def process_batch_original(batch, tta=False):
    if tta:
        fliped = hflip_batch(batch)
        feat_orig = learner.model.get_original_feature(batch.to(conf.device))
        feat_flip = learner.model.get_original_feature(fliped.to(conf.device))
        feat = (feat_orig + feat_flip) / 2
    else:
        feat = learner.model.get_original_feature(batch.to(conf.device))
    return feat


def process_batch_xcos(batch, tta=False):
    if tta:
        fliped = hflip_batch(batch)
        flattened_feature1, feat_map1 = learner.model(batch.to(conf.device))
        flattened_feature2, feat_map2 = learner.model(fliped.to(conf.device))
        feat_map = (feat_map1 + feat_map2) / 2
        flattened_feature = (flattened_feature1 + flattened_feature2) / 2
    else:
        flattened_feature, feat_map = learner.model(batch.to(conf.device))
    return flattened_feature, feat_map

## Init Learner and conf

In [14]:
conf = get_config(training=False)
conf.batch_size=20 # Why bs_size can only be the number that divide 6000 well?
learner = face_learner(conf, inference=True)

{'detachAttentionGradient': True, 'fixed_str': 'ir_se50.pth', 'pretrainedMdl': 'ir_se50.pth', 'data_path': PosixPath('data'), 'work_path': PosixPath('work_space'), 'model_path': PosixPath('work_space/models'), 'log_path': PosixPath('work_space/log'), 'save_path': PosixPath('work_space/save'), 'exp_title': 'xCos', 'exp_comment': 'fromScratch_CosFace', 'input_size': [112, 112], 'embedding_size': 1568, 'use_mobilfacenet': False, 'modelType': 'ArcFace', 'net_depth': 50, 'drop_ratio': 0.6, 'net_mode': 'ir_se', 'device': device(type='cuda', index=0), 'test_transform': Compose(
    ToTensor()
    Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
), 'data_mode': 'emore', 'vgg_folder': PosixPath('data/faces_vgg_112x112'), 'ms1m_folder': PosixPath('data/faces_ms1m_112x112'), 'emore_folder': PosixPath('data/faces_emore'), 'batch_size': 20, 'USE_SOFTMAX': True, 'SOFTMAX_T': 1, 'facebank_path': PosixPath('data/facebank'), 'threshold': 1.5, 'threshold_xCos': 0.2338, 'face_limit': 10, 'min_face_si

# Process IJB-B data and save them

In [15]:
loader = torch.utils.data.DataLoader(
    IJBCroppedFacesDataset('data/IJB_release/IJBB'),
    batch_size=1
)

## Original model

In [16]:
learner.load_state(conf, 'ir_se50.pth', model_only=True, from_save_folder=True, strict=False, model_atten=False)
learner.model.eval()
learner.model.returnGrid = True  # Remember to reset this before return!
learner.model_attention.eval()

dst_dir = 'work_space/IJB_features/IJBB_ir_se50'
os.makedirs(op.join(dst_dir), exist_ok=True)
os.makedirs(op.join(dst_dir, 'loose_crop'), exist_ok=True)
with torch.no_grad():
    for i, batch in enumerate(loader):
        tensor = batch['tensor']
        src_path = batch['path']
        if i % 200 == 0:
            print(f'Processing {src_path[0]}')
        target_path = op.join(dst_dir, *src_path[0].split('/')[-2:]) + '.npy'
        if op.exists(target_path):
            if i % 200 == 0:
                print(f"Skipping {target_path} because it exists.")
            continue
        feat = process_batch_original(tensor, tta=True)
        np.save(target_path, feat.cpu().numpy())

Processing data/IJB_release/IJBB/loose_crop/1.jpg
Skipping work_space/IJB_features/IJBB_ir_se50/loose_crop/1.jpg.npy because it exists.
Processing data/IJB_release/IJBB/loose_crop/201.jpg
Skipping work_space/IJB_features/IJBB_ir_se50/loose_crop/201.jpg.npy because it exists.
Processing data/IJB_release/IJBB/loose_crop/401.jpg
Skipping work_space/IJB_features/IJBB_ir_se50/loose_crop/401.jpg.npy because it exists.
Processing data/IJB_release/IJBB/loose_crop/601.jpg
Skipping work_space/IJB_features/IJBB_ir_se50/loose_crop/601.jpg.npy because it exists.
Processing data/IJB_release/IJBB/loose_crop/801.jpg
Skipping work_space/IJB_features/IJBB_ir_se50/loose_crop/801.jpg.npy because it exists.
Processing data/IJB_release/IJBB/loose_crop/1001.jpg
Skipping work_space/IJB_features/IJBB_ir_se50/loose_crop/1001.jpg.npy because it exists.
Processing data/IJB_release/IJBB/loose_crop/1201.jpg
Skipping work_space/IJB_features/IJBB_ir_se50/loose_crop/1201.jpg.npy because it exists.
Processing data/IJB_

Processing data/IJB_release/IJBB/loose_crop/11601.jpg
Skipping work_space/IJB_features/IJBB_ir_se50/loose_crop/11601.jpg.npy because it exists.
Processing data/IJB_release/IJBB/loose_crop/11801.jpg
Skipping work_space/IJB_features/IJBB_ir_se50/loose_crop/11801.jpg.npy because it exists.
Processing data/IJB_release/IJBB/loose_crop/12001.jpg
Skipping work_space/IJB_features/IJBB_ir_se50/loose_crop/12001.jpg.npy because it exists.
Processing data/IJB_release/IJBB/loose_crop/12201.jpg
Skipping work_space/IJB_features/IJBB_ir_se50/loose_crop/12201.jpg.npy because it exists.
Processing data/IJB_release/IJBB/loose_crop/12401.jpg
Skipping work_space/IJB_features/IJBB_ir_se50/loose_crop/12401.jpg.npy because it exists.
Processing data/IJB_release/IJBB/loose_crop/12601.jpg
Skipping work_space/IJB_features/IJBB_ir_se50/loose_crop/12601.jpg.npy because it exists.
Processing data/IJB_release/IJBB/loose_crop/12801.jpg
Skipping work_space/IJB_features/IJBB_ir_se50/loose_crop/12801.jpg.npy because it 

Processing data/IJB_release/IJBB/loose_crop/23001.jpg
Skipping work_space/IJB_features/IJBB_ir_se50/loose_crop/23001.jpg.npy because it exists.
Processing data/IJB_release/IJBB/loose_crop/23201.jpg
Skipping work_space/IJB_features/IJBB_ir_se50/loose_crop/23201.jpg.npy because it exists.
Processing data/IJB_release/IJBB/loose_crop/23401.jpg
Skipping work_space/IJB_features/IJBB_ir_se50/loose_crop/23401.jpg.npy because it exists.
Processing data/IJB_release/IJBB/loose_crop/23601.jpg
Skipping work_space/IJB_features/IJBB_ir_se50/loose_crop/23601.jpg.npy because it exists.
Processing data/IJB_release/IJBB/loose_crop/23801.jpg
Skipping work_space/IJB_features/IJBB_ir_se50/loose_crop/23801.jpg.npy because it exists.
Processing data/IJB_release/IJBB/loose_crop/24001.jpg
Skipping work_space/IJB_features/IJBB_ir_se50/loose_crop/24001.jpg.npy because it exists.
Processing data/IJB_release/IJBB/loose_crop/24201.jpg
Skipping work_space/IJB_features/IJBB_ir_se50/loose_crop/24201.jpg.npy because it 

Processing data/IJB_release/IJBB/loose_crop/34401.jpg
Skipping work_space/IJB_features/IJBB_ir_se50/loose_crop/34401.jpg.npy because it exists.
Processing data/IJB_release/IJBB/loose_crop/34601.jpg
Skipping work_space/IJB_features/IJBB_ir_se50/loose_crop/34601.jpg.npy because it exists.
Processing data/IJB_release/IJBB/loose_crop/34801.jpg
Skipping work_space/IJB_features/IJBB_ir_se50/loose_crop/34801.jpg.npy because it exists.
Processing data/IJB_release/IJBB/loose_crop/35001.jpg
Skipping work_space/IJB_features/IJBB_ir_se50/loose_crop/35001.jpg.npy because it exists.
Processing data/IJB_release/IJBB/loose_crop/35201.jpg
Skipping work_space/IJB_features/IJBB_ir_se50/loose_crop/35201.jpg.npy because it exists.
Processing data/IJB_release/IJBB/loose_crop/35401.jpg
Skipping work_space/IJB_features/IJBB_ir_se50/loose_crop/35401.jpg.npy because it exists.
Processing data/IJB_release/IJBB/loose_crop/35601.jpg
Skipping work_space/IJB_features/IJBB_ir_se50/loose_crop/35601.jpg.npy because it 

Processing data/IJB_release/IJBB/loose_crop/45801.jpg
Skipping work_space/IJB_features/IJBB_ir_se50/loose_crop/45801.jpg.npy because it exists.
Processing data/IJB_release/IJBB/loose_crop/46001.jpg
Skipping work_space/IJB_features/IJBB_ir_se50/loose_crop/46001.jpg.npy because it exists.
Processing data/IJB_release/IJBB/loose_crop/46201.jpg
Skipping work_space/IJB_features/IJBB_ir_se50/loose_crop/46201.jpg.npy because it exists.
Processing data/IJB_release/IJBB/loose_crop/46401.jpg
Skipping work_space/IJB_features/IJBB_ir_se50/loose_crop/46401.jpg.npy because it exists.
Processing data/IJB_release/IJBB/loose_crop/46601.jpg
Skipping work_space/IJB_features/IJBB_ir_se50/loose_crop/46601.jpg.npy because it exists.
Processing data/IJB_release/IJBB/loose_crop/46801.jpg
Skipping work_space/IJB_features/IJBB_ir_se50/loose_crop/46801.jpg.npy because it exists.
Processing data/IJB_release/IJBB/loose_crop/47001.jpg
Skipping work_space/IJB_features/IJBB_ir_se50/loose_crop/47001.jpg.npy because it 

...

Processing data/IJB_release/IJBB/loose_crop/68601.jpg
Skipping work_space/IJB_features/IJBB_ir_se50/loose_crop/68601.jpg.npy because it exists.
Processing data/IJB_release/IJBB/loose_crop/68801.jpg
Skipping work_space/IJB_features/IJBB_ir_se50/loose_crop/68801.jpg.npy because it exists.
Processing data/IJB_release/IJBB/loose_crop/69001.jpg
Skipping work_space/IJB_features/IJBB_ir_se50/loose_crop/69001.jpg.npy because it exists.
Processing data/IJB_release/IJBB/loose_crop/69201.jpg
Skipping work_space/IJB_features/IJBB_ir_se50/loose_crop/69201.jpg.npy because it exists.
Processing data/IJB_release/IJBB/loose_crop/69401.jpg
Skipping work_space/IJB_features/IJBB_ir_se50/loose_crop/69401.jpg.npy because it exists.
Processing data/IJB_release/IJBB/loose_crop/69601.jpg
Skipping work_space/IJB_features/IJBB_ir_se50/loose_crop/69601.jpg.npy because it exists.
Processing data/IJB_release/IJBB/loose_crop/69801.jpg
Skipping work_space/IJB_features/IJBB_ir_se50/loose_crop/69801.jpg.npy because it 

Processing data/IJB_release/IJBB/loose_crop/80001.jpg
Skipping work_space/IJB_features/IJBB_ir_se50/loose_crop/80001.jpg.npy because it exists.
Processing data/IJB_release/IJBB/loose_crop/80201.jpg
Skipping work_space/IJB_features/IJBB_ir_se50/loose_crop/80201.jpg.npy because it exists.
Processing data/IJB_release/IJBB/loose_crop/80401.jpg
Skipping work_space/IJB_features/IJBB_ir_se50/loose_crop/80401.jpg.npy because it exists.
Processing data/IJB_release/IJBB/loose_crop/80601.jpg
Skipping work_space/IJB_features/IJBB_ir_se50/loose_crop/80601.jpg.npy because it exists.
Processing data/IJB_release/IJBB/loose_crop/80801.jpg
Skipping work_space/IJB_features/IJBB_ir_se50/loose_crop/80801.jpg.npy because it exists.
Processing data/IJB_release/IJBB/loose_crop/81001.jpg
Skipping work_space/IJB_features/IJBB_ir_se50/loose_crop/81001.jpg.npy because it exists.
Processing data/IJB_release/IJBB/loose_crop/81201.jpg
Skipping work_space/IJB_features/IJBB_ir_se50/loose_crop/81201.jpg.npy because it 

Processing data/IJB_release/IJBB/loose_crop/91401.jpg
Skipping work_space/IJB_features/IJBB_ir_se50/loose_crop/91401.jpg.npy because it exists.
Processing data/IJB_release/IJBB/loose_crop/91601.jpg
Skipping work_space/IJB_features/IJBB_ir_se50/loose_crop/91601.jpg.npy because it exists.
Processing data/IJB_release/IJBB/loose_crop/91801.jpg
Skipping work_space/IJB_features/IJBB_ir_se50/loose_crop/91801.jpg.npy because it exists.
Processing data/IJB_release/IJBB/loose_crop/92001.jpg
Skipping work_space/IJB_features/IJBB_ir_se50/loose_crop/92001.jpg.npy because it exists.
Processing data/IJB_release/IJBB/loose_crop/92201.jpg
Skipping work_space/IJB_features/IJBB_ir_se50/loose_crop/92201.jpg.npy because it exists.
Processing data/IJB_release/IJBB/loose_crop/92401.jpg
Skipping work_space/IJB_features/IJBB_ir_se50/loose_crop/92401.jpg.npy because it exists.
Processing data/IJB_release/IJBB/loose_crop/92601.jpg
Skipping work_space/IJB_features/IJBB_ir_se50/loose_crop/92601.jpg.npy because it 

Processing data/IJB_release/IJBB/loose_crop/102801.jpg
Skipping work_space/IJB_features/IJBB_ir_se50/loose_crop/102801.jpg.npy because it exists.
Processing data/IJB_release/IJBB/loose_crop/103001.jpg
Skipping work_space/IJB_features/IJBB_ir_se50/loose_crop/103001.jpg.npy because it exists.
Processing data/IJB_release/IJBB/loose_crop/103201.jpg
Skipping work_space/IJB_features/IJBB_ir_se50/loose_crop/103201.jpg.npy because it exists.
Processing data/IJB_release/IJBB/loose_crop/103401.jpg
Skipping work_space/IJB_features/IJBB_ir_se50/loose_crop/103401.jpg.npy because it exists.
Processing data/IJB_release/IJBB/loose_crop/103601.jpg
Skipping work_space/IJB_features/IJBB_ir_se50/loose_crop/103601.jpg.npy because it exists.
Processing data/IJB_release/IJBB/loose_crop/103801.jpg
Skipping work_space/IJB_features/IJBB_ir_se50/loose_crop/103801.jpg.npy because it exists.
Processing data/IJB_release/IJBB/loose_crop/104001.jpg
Skipping work_space/IJB_features/IJBB_ir_se50/loose_crop/104001.jpg.n

KeyboardInterrupt: 

## Our model

In [ ]:
def run_ours_IJBC(model_name):
    learner.load_state(
        conf, f'{model_name}.pth',
        model_only=True, from_save_folder=True, strict=True, model_atten=True)
    learner.model.eval()
    learner.model.returnGrid = True  # Remember to reset this before return!
    learner.model_attention.eval()

    dst_dir = f'work_space/IJB_features/IJBB_{model_name}/'
    os.makedirs(op.join(dst_dir), exist_ok=True)
    os.makedirs(op.join(dst_dir, 'loose_crop'), exist_ok=True)

    with torch.no_grad():
        for i, batch in enumerate(loader):
            tensor = batch['tensor']
            src_path = batch['path']
            target_path = op.join(dst_dir, *src_path[0].split('/')[-2:]) + '.npz'
            if i % 2000 == 0:
                print(f'Processing {src_path[0]} \nSaving to {target_path}')
            if op.exists(target_path):
                if i % 2000 == 0:
                    print(f"Skipping {target_path} because it exists.")
                continue
            flattened_feature, feat_map = process_batch_xcos(tensor, tta=True)
            np.savez(target_path, flattened_feature=flattened_feature.cpu().numpy(),
                     feat_map=feat_map.cpu().numpy())


model_names = [
#     '2019-09-01-15-30_accuracy:0.9946666666666667_step:218346_CosFace',
    '2019-09-02-08-21_accuracy:0.9968333333333333_step:436692_CosFace',
#     '2019-08-25-14-35_accuracy:0.9931666666666666_step:218349_None',
    '2019-08-30-07-36_accuracy:0.9953333333333333_step:655047_None'
]
for model_name in model_names:
    print(f'-------- Runing on model {model_name} --------')
    run_ours_IJBC(model_name)